По традиции начинаем с вопроса....

У нас есть множество различных новостных статей на абсолютно разные темы. Нам нужно определить тему статьи (узкую) (найти в массе из 3000+ знаков какие-то самые значимые для этой статьи слова, которые делают её отличной от других статей и которые могут поступить в заголовок и составлять её  краткий пересказ) --- как нам это сделать? 


Очевидным образом, не все слова were created equal. Мы уже знакомы с рядос стоп слов, которые мы в лингвистике яростно выбрасываем из предложения, однако, и другие слова в тексте неравнознчны.......

# TF_IDF

Что это? И зачем оно нужно? 

По сути, это метрика, которая показывает насколько важно/релевантно слово в заданном документе. используется для анализа темы и в поисковиках.

В этом семинаре мы попытаемся создать с помощью неё наш собственный поисковик.

Представим, что нам на вход в поисковике поступило 'a brown cow'. У нас в мировой паутине лежит куча-куча-куча документов с множеством различных слов. Как по заданным словам понять, какой из них релевантнее? 

In [1]:
import pandas as pd # если нет pandas - не страшно - он тут для красоты
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
inquery = 'the brown cows'

documentA = 'the black man went out at night for a stroll past the brown cows he owned He also noticed a wolf on the stroll but did not pay attention ...'
documentB = 'The brown children sat around the fire and sang the songs ...'
documentC = 'Everything about brown cows.  Brown cows live in a harsh enviroment ... '
documentD = 'Everything about cows. Cows live in different enviroments ...'

Как мы можем, не залезая в семантику, узнать, какой текст выдать первым?

tf


idf


term frequency–inverse document frequency

In [44]:

import nltk
nltk.download('punkt')
bagOfWordsA = nltk.word_tokenize(documentA)
bagOfWordsB = nltk.word_tokenize(documentB)
bagOfWordsC = nltk.word_tokenize(documentC)
bagOfWordsD = nltk.word_tokenize(documentD)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avorl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
uniqueWords = set(bagOfWordsA).union(set(bagOfWordsB)).union(set(bagOfWordsC)).union(set(bagOfWordsD))

In [46]:
uniqueWords

{',',
 '.',
 '...',
 'Brown',
 'Cows',
 'Everything',
 'He',
 'The',
 'a',
 'about',
 'also',
 'ans',
 'around',
 'at',
 'attention',
 'black',
 'brown',
 'but',
 'children',
 'cows',
 'did',
 'different',
 'enviroment',
 'enviroments',
 'fire',
 'for',
 'harsh',
 'in',
 'live',
 'man',
 'night',
 'not',
 'noticed',
 'on',
 'out',
 'past',
 'pay',
 'sang',
 'sat',
 'songs',
 'stroll',
 'the',
 'went',
 'wolf'}

In [47]:
numOfWordsA = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsA:
    numOfWordsA[word] += 1

numOfWordsB = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsB:
    numOfWordsB[word] += 1
    
numOfWordsC = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsC:
    numOfWordsC[word] += 1
    
numOfWordsD = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsD:
    numOfWordsD[word] += 1
    
df = pd.DataFrame([numOfWordsA, numOfWordsB,numOfWordsC, numOfWordsD ])    

In [48]:
pd.set_option('display.max_columns', None)
df

,noticed,...,.,did,Cows,not,a,He,stroll,fire,past,Brown,different,on,pay,out,around,Everything,attention,the,enviroment,black,man,night,harsh,cows,also,wolf,sang,enviroments,The,songs,children,live,in,went,",",about,but,for,sat,ans,brown,at
0,1,1,0,1,0,1,4,1,2,0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,0,1,1,1,0,0,0,0,0,0,0,1,1,0,1,1,0,0,1,1
1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,1,1,1,0
2,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,2,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0
3,0,1,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0


Почему иногда всё же но стоит выкидывать стоп-слова - предлоги, артикли и союзы?


## TF

Tf --- term frequency. В самом первом упрощении - это то, сколько раз слово встретилось в тексте (чем чаще искомое слово встретилось в заданном тексте по сравнению с другими, тем этот текст релевантнее запросу), в чём тут проблема? 

Поэтому есть всякие adjustments. Так часто tf слова в документе считается как количество раз, когда оно появилось в этом документе, деленное на общее число слов документе.

In [ ]:
def computeTF(wordDict, bagOfWords):
    #напишите функцию, которая получает на ввход bagofwords, а возвращает словарь с tf-ами для каждого слова 
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict